In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os
from glob import glob

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
BASE_TREE = Polygon(zip(TX, TY))

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

def calculate_score(df):
    total_score = 0
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            return None
        all_coords = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            coords = np.array(poly.exterior.coords)
            all_coords.append(coords)
        all_coords = np.vstack(all_coords)
        x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
        y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
        side = max(x_range, y_range)
        total_score += side**2 / n
    return total_score

print("Functions defined")

In [ ]:
# Check all CSV files in snapshots
files_to_check = [
    '/home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/submission_best.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/experiments/005_backward_propagation/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/experiments/005_backward_propagation/optimized.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv',
]

results = []
for filepath in files_to_check:
    try:
        df = pd.read_csv(filepath)
        if len(df) == 20100:
            score = calculate_score(df)
            if score:
                results.append((filepath, score))
                print(f"{os.path.basename(filepath)}: {score:.6f}")
    except Exception as e:
        print(f"Error with {filepath}: {e}")

results.sort(key=lambda x: x[1])
print("\nBest files:")
for f, s in results[:5]:
    print(f"  {s:.6f}: {os.path.basename(f)}")

In [ ]:
# Check more snapshots for better solutions
import subprocess

# Find all CSV files with 20101 lines
result = subprocess.run(
    ['find', '/home/nonroot/snapshots', '-name', '*.csv', '-size', '+1M'],
    capture_output=True, text=True
)
all_csvs = result.stdout.strip().split('\n')

print(f"Found {len(all_csvs)} CSV files")

# Check each one
best_score = 70.676102
best_file = None

for filepath in all_csvs:
    if not filepath:
        continue
    try:
        df = pd.read_csv(filepath)
        if len(df) == 20100:
            score = calculate_score(df)
            if score and score < best_score:
                best_score = score
                best_file = filepath
                print(f"NEW BEST: {score:.6f} from {filepath}")
    except:
        pass

print(f"\nBest score found: {best_score:.6f}")
if best_file:
    print(f"Best file: {best_file}")